# Working with functions in Azure OpenAI
This notebook shows how to use the Chat Completions API in combination with functions to extend the current capabilities of GPT models. GPT models, do not inherently support real-time interaction with external systems, databases, or files. However, functions can be used to do so.

Overview: <br>
`functions` is an optional parameter in the Chat Completion API which can be used to provide function specifications. This allows models to generate function arguments for the specifications provided by the user. 

Note: The API will not execute any function calls. Executing function calls using the outputed argments must be done by developers. 

## Setup

In [ ]:
import os
import openai
from openai import OpenAI
import json
from dotenv import load_dotenv


# Load environment variables
if load_dotenv():
    print("Found OpenAPI Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("No file .env found")

# Setting up the deployment name
deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")

# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") 

# Currently Chat Completion API have the following versions available: 2023-07-01-preview
openai.api_version = os.getenv("OPENAI_API_VERSION") 

from openai import AzureOpenAI
 
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),
  api_version="2023-12-01-preview",
  azure_deployment = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
)

## 1.0 Test functions

This code calls the model with the user query and the set of functions (or tools) defined in the tools parameter. The model then can choose if it calls a function. If a function is called, the content will be in a strigified JSON object. The function call that should be made and arguments are location in:  response[`choices`][0][`tools_calls`].

In [2]:
# Example function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

# Define the functions to use
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

def get_function_call(messages, tool_choice = "auto"):
    

    # Call the model with the user query (messages) and the functions defined in the functions parameter
    response = client.chat.completions.create(
        model = deployment_name,
        messages = messages,
        tools = tools,
        tool_choice = tool_choice, 
    )

    return response.choices[0].message

### Forcing the use of a specific function or no function
By changing the value of the `tools` parameter you can allow the model to decide what function to use, force the model to use a specific function, or force the model to use no function.

In [ ]:
first_message = [{"role": "user", "content": "What's the weather like in San Francisco?"}]
# 'auto' : Let the model decide what function to call
print("Let the model decide what function to call:")
response_message=get_function_call(first_message, "auto")
print(response_message)

tool_calls = response_message.tool_calls
# Step 2: check if the model wanted to call a function
if tool_calls:
    print("it decided to call the following function:")
    print(tool_calls)
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
    }  # only one function in this example, but you can have multiple
    first_message.append(response_message)  # extend conversation with assistant's reply
    # Step 4: send the info for each function call and function response to the model
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
        first_message.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model = deployment_name,
            messages = first_message
        )
        print("The model responds with the following message:")
        print(second_response.choices[0].message)
    # get a new response from the model where it can see the function response


# 'none' : Don't call any function 
#print("Don't call any function:")
#print(get_function_call(first_message, "none")["choices"][0]['message'])

# force a specific function call
#print("Force a specific function call:")
#print(get_function_call(first_message, function_call={"name": "get_current_weather"})["choices"][0]['message'])

## 2.0 Defining functions
Now that we know how to work with functions, let's define some functions in code so that we can walk through the process of using functions end to end.

### Function #1: Get current time

In [4]:
import pytz
from datetime import datetime

def get_current_time(location):
    try:
        # Get the timezone for the city
        timezone = pytz.timezone(location)

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%I:%M:%S %p")

        return current_time
    except:
        return "Sorry, I couldn't find the timezone for that location."

In [ ]:
get_current_time("America/New_York")

### Function #2: Get stock market data
For simplicity, we're just hard coding some stock market data but you could easily edit the code to call out to an API to retrieve real-time data.

In [6]:
import yfinance as yf
from datetime import datetime, timedelta

def get_current_stock_price(name):
    """Method to get current stock price"""
    ticker_data = yf.Ticker(name)
    recent = ticker_data.history(period='1d')
    return str(recent.iloc[0]['Close']) + ' USD'

In [ ]:
print(get_current_stock_price("MSFT"))

### Function #3: Calculator 

In [8]:
import math

def calculator(num1, num2, operator):
    if operator == '+':
        return str(num1 + num2)
    elif operator == '-':
        return str(num1 - num2)
    elif operator == '*':
        return str(num1 * num2)
    elif operator == '/':
        return str(num1 / num2)
    elif operator == '**':
        return str(num1 ** num2)
    elif operator == 'sqrt':
        return str(math.sqrt(num1))
    else:
        return "Invalid operator"

In [ ]:
print(calculator(5, 5, '+'))

## 3.0 Calling a function using GPT

Steps for Function Calling: 

1. Call the model with the user query and a set of functions defined in the functions parameter.
2. The model can choose to call a function; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may generate invalid JSON or hallucinate parameters).
3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

### 3.1 Describe the functions so that the model knows how to call them

In [10]:
functions = [
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "Get the current time in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The location name. The pytz is used to get the timezone for that location. Location names should be in a format like America/New_York, Asia/Bangkok, Europe/London",
                        }
                    },
                    "required": ["location"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_current_stock_price",
                "description": "Get the stock value for a given stock name",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "The stock name. The stock market symbol name is used to retrieve the value on the stock exchange"
                        },
                    },
                    "required": ["name"],
                },    
            },
        },
        {
            "type": "function",
            "function": {
                "name": "calculator",
                "description": "A simple calculator used to perform basic arithmetic operations",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "num1": {"type": "number"},
                        "num2": {"type": "number"},
                        "operator": {"type": "string", "enum": ["+", "-", "*", "/", "**", "sqrt"]},
                    },
                    "required": ["num1", "num2", "operator"],
                },
            },
        }
    ]

available_functions = {
            "get_current_time": get_current_time,
            "get_current_stock_price": get_current_stock_price,
            "calculator": calculator,
        } 

### 3.2 Define a helper function to validate the function call
It's possible that the models could generate incorrect function calls so it's important to validate the calls. Here we define a simple helper function to validate the function call although you could apply more complex validation for your use case.

In [11]:
import inspect

# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided 
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [12]:
def run_conversation(messages, functions, available_functions, deployment_id):
    # Step 1: send the conversation and available functions to GPT

    response = client.chat.completions.create(
        model = deployment_name,
        messages = messages,
        tools = functions,
        tool_choice = "auto", 
    )
    print(response)
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    # Step 2: check if GPT wanted to call a function
    if tool_calls:
        print("Recommended Function call:")
        print(tool_calls)
        print()
    
        # Step 3: call the function
        messages.append(response_message)

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            # verify function exists
            if function_name not in available_functions:
                return "Function " + function_name + " does not exist"
            else:
                print("Calling function: " + function_name)
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            ) 
            print("Addding this message to the next prompt:")
            print(messages)
             # extend conversation with function response
            second_response = client.chat.completions.create(
                model = deployment_id,
                messages = messages)  # get a new response from the model where it can see the function response
            return second_response

In [ ]:
messages = [{"role": "user", "content": "What time is it in New York?"}]
assistant_response = run_conversation(messages, functions, available_functions, deployment_name)
print("The model responds with the function data:")
print(assistant_response.choices[0].message)

In [ ]:
messages = [{"role": "user", "content": "What is the value of the Microsoft stock?"}]
assistant_response = run_conversation(messages, functions, available_functions, deployment_name)
print("The model responds with the function data:")
print(assistant_response.choices[0].message)